In [1]:
import requests
import json
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import datetime as dt

# B. Khám phá dữ liệu

## 1. Đọc dữ liệu, tính số dòng và số cột

Đọc dữ liệu từ file `weather-2021.csv` và in ra 5 dòng đầu tiên

In [2]:
df = pd.read_csv('weather-2021.csv')
df.head()

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,VVTS,observation,1609441200,VVTS,Ho Chi Minh City,1609434000,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VVTS,observation,1609443000,VVTS,Ho Chi Minh City,1609435800,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VVTS,observation,1609444800,VVTS,Ho Chi Minh City,1609437600,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VVTS,observation,1609446600,VVTS,Ho Chi Minh City,1609439400,N,73,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VVTS,observation,1609448400,VVTS,Ho Chi Minh City,1609441200,N,73,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tính số dòng, số cột và lưu vào 2 biến `num_rows` và `num_cols`

In [3]:
num_rows, num_cols = df.shape
print(f'Number of rows: {num_rows}\nNumber of columns: {num_cols}')

Number of rows: 17390
Number of columns: 45


## 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong tập dữ liệu là thông tin và các chỉ số về thời tiết của quận Tân Bình (TP.HCM) tại thời điểm nhất định (cập nhật 30 phút/lần). Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau, tức không có dòng nào bị 'lạc loài'

## 3. Dữ liệu có các dòng bị lặp không?

Ta kiểm tra xem có dòng nào bị lặp không bằng cách sử dụng các phương thức `duplicated()` và `any()` và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu không có dòng bị lặp

In [4]:
have_duplicate_rows = df.duplicated().any()
have_duplicate_rows

False

Như vậy, dữ liệu không có dòng nào bị lặp lặp 

## 4. Tỉ lệ giá trị thiếu của từng cột

Ta tính tỉ lệ giá trị thiếu của từng cột. Đầu tiên ta dùng `isnull()` để biết được các giá trị thiếu, sau đó dùng `sum()` để tính tổng số giá trị thiếu theo từng cột. Cuối cùng ta chia cho tổng số dòng để tính tỉ lệ giá trị thiếu của mỗi cột và lưu kết quả vào biến `missing_ratio`

In [5]:
missing_ratio = df.isnull().sum()
missing_ratio = missing_ratio / num_rows
missing_ratio

key                          0.000000
class                        0.000000
expire_time_gmt              0.000000
obs_id                       0.000000
obs_name                     0.000000
valid_time_gmt               0.000000
day_ind                      0.000000
temp                         0.000000
wx_icon                      0.000000
icon_extd                    0.000000
wx_phrase                    0.000000
pressure_tend                1.000000
pressure_desc                1.000000
dewPt                        0.000115
heat_index                   0.000115
rh                           0.000115
pressure                     0.000575
vis                          0.000000
wc                           0.000000
wdir                         0.255952
wdir_cardinal                0.000000
gust                         0.994767
wspd                         0.000000
max_temp                     1.000000
min_temp                     1.000000
precip_total                 1.000000
precip_hrly 

Có thể thấy, có những cột mà tỉ lệ giá trị thiếu lên đến hơn 99% hoặc thậm chí là 100%

### Loại bỏ một số cột không cần thiết

Ta có thể thấy có rất nhiều cột không có giá trị (hay tỉ lệ dữ liệu bị thiếu xấp xỉ 100%). Để làm gọn dữ liệu và để dễ quản lý dataframe hơn, ta sẽ bỏ đi những cột không có ý nghĩa đó

In [6]:
del_cols = []
for i in range(len(missing_ratio)):
    if missing_ratio[i] > 0.9:
        del_cols.append(missing_ratio.index[i])
df = df.drop(del_cols,axis=1)
df.columns

Index(['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name',
       'valid_time_gmt', 'day_ind', 'temp', 'wx_icon', 'icon_extd',
       'wx_phrase', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc',
       'wdir', 'wdir_cardinal', 'wspd', 'uv_desc', 'feels_like', 'uv_index',
       'clds'],
      dtype='object')

Sau khi bỏ đi những cột có giá trị thiếu nhiều thì bây giờ ta nên điền vào những giá trị thiếu đó là những giá trị trung bình của mỗi cột tương ứng với từng điều kiện thời tiết

Đầu tiên là lấy tên những cột có giá trị thiếu và lưu vào `missing_cols`

In [7]:
missing_cols = df.columns[df.isnull().any()]

Tiếp theo là tính giá trị trung bình của mỗi cột trong `df` theo các giá trị trong `wx_phrase` (điều kiện thời tiết)

In [8]:
mean = df.groupby('wx_phrase').mean()
mean

C:\Users\Admin\AppData\Local\Temp\ipykernel_16700\2091699250.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean = df.groupby('wx_phrase').mean()


,expire_time_gmt,valid_time_gmt,temp,wx_icon,icon_extd,dewPt,heat_index,rh,pressure,vis,wc,wdir,wspd,feels_like,uv_index
wx_phrase,,,,,,,,,,,,,,,
Cloudy,1.631486e+09,1.631479e+09,79.962963,26.000000,2600.000000,73.814815,83.907407,82.222222,29.765000,5.111111,79.962963,216.190476,5.481481,83.907407,1.500000
Fair,1.619870e+09,1.619863e+09,82.007961,33.479486,3347.948561,71.213411,86.613595,71.656767,29.795993,5.625842,82.007961,158.130277,6.134415,86.613595,1.859767
Fog,1.625648e+09,1.625640e+09,77.925926,20.000000,2000.000000,75.209877,80.407407,92.123457,29.814444,1.554691,77.925926,230.000000,2.135802,80.407407,0.777778
Haze,1.615808e+09,1.615801e+09,84.750000,21.000000,2100.000000,67.625000,86.625000,58.875000,29.805000,1.991250,84.750000,260.000000,4.625000,86.625000,7.000000
Heavy Rain Shower,1.630871e+09,1.630864e+09,78.444444,40.000000,1102.000000,77.000000,82.222222,95.555556,29.733333,1.472222,78.444444,257.142857,12.000000,82.222222,3.666667
Heavy Rain Shower / Windy,1.627074e+09,1.627067e+09,76.000000,40.000000,1192.000000,75.000000,76.500000,97.000000,29.685000,1.000000,76.000000,295.000000,25.500000,76.500000,5.000000
Heavy T-Storm,1.626596e+09,1.626589e+09,76.571429,4.000000,402.000000,75.250000,78.678571,95.964286,29.734286,1.232500,76.571429,200.370370,11.535714,78.678571,1.357143
Heavy T-Storm / Windy,1.625373e+09,1.625366e+09,74.000000,4.000000,492.000000,73.666667,74.166667,99.000000,29.715000,0.640000,74.000000,236.666667,24.500000,74.166667,2.666667
Light Rain,1.629199e+09,1.629191e+09,78.528713,11.000000,1201.000000,76.835644,82.007921,94.938614,29.730297,4.544554,78.528713,216.178161,4.712871,82.007921,0.657426


Cuối cùng là điền vào các giá trị thiếu giá trị trung bình của mỗi cột ứng với mỗi giá trị trong cột `wx_phrase` (điều kiện thời tiết)

In [9]:
for col in missing_cols:
    df[col] = df[col].fillna(df['wx_phrase'].map(mean[col]))

Xem số giá trị thiếu trong các cột

In [10]:
df.isnull().sum()

key                0
class              0
expire_time_gmt    0
obs_id             0
obs_name           0
valid_time_gmt     0
day_ind            0
temp               0
wx_icon            0
icon_extd          0
wx_phrase          0
dewPt              0
heat_index         0
rh                 0
pressure           0
vis                0
wc                 0
wdir               0
wdir_cardinal      0
wspd               0
uv_desc            0
feels_like         0
uv_index           0
clds               0
dtype: int64

## 5. Mỗi cột có ý nghĩa gì?

Dựa trên thông tin tập dữ liệu đã mô tả ở phần Thu thập dữ liệu, ta chọn ra những cột liên quan và cần thiết:
- Thời điểm biểu diễn các chỉ số thời tiết: 'valid_time_gmt'
- Thông tin về chỉ số thời tiết: 'temp', 'wx_phrase', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wspd', 'uv_desc', 'feels_like', 'uv_index'

Như vậy các cột bị loại bỏ bao gồm: 'key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name', 'day_ind', 'wx_icon', 'icon_extd', 'wc', 'wdir', 'wdir_cardinal', 'clds'

### Loại bỏ một số cột không cần thiết

In [11]:
del_cols = ['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name', 'day_ind', 'wx_icon', 'icon_extd', 'wc', 'wdir', 'wdir_cardinal', 'clds']
df = df.drop(del_cols, axis=1)
df.columns

Index(['valid_time_gmt', 'temp', 'wx_phrase', 'dewPt', 'heat_index', 'rh',
       'pressure', 'vis', 'wspd', 'uv_desc', 'feels_like', 'uv_index'],
      dtype='object')

Để dễ dàng ghi nhớ ý nghĩa của từng cột, ta tiến hành thay đổi tên cột bằng phương thức `rename()`

In [12]:
df.rename(columns = {'valid_time_gmt':'Time',
                     'temp':'Temperature',
                     'wx_phrase':'Condition',
                     'dewPt':'Dew Point',
                     'heat_index':'Heat Index',
                     'rh':'Humidity',
                     'pressure':'Pressure',
                     'vis':'Wind Force',
                     'wspd':'Wind Speed',
                     'uv_desc':'UV Description',
                     'feels_like':'Temperature Feels Like',
                     'uv_index':'UV Index'}, inplace = True)
# Sắp xếp lại thứ tự các cột
df = df[['Time',
         'Temperature', 'Heat Index', 'Temperature Feels Like',
         'Dew Point', 'Humidity',
         'Wind Force', 'Wind Speed',
         'Pressure',
         'UV Description', 'UV Index',
         'Condition',]]
df.columns

Index(['Time', 'Temperature', 'Heat Index', 'Temperature Feels Like',
       'Dew Point', 'Humidity', 'Wind Force', 'Wind Speed', 'Pressure',
       'UV Description', 'UV Index', 'Condition'],
      dtype='object')

### Thống kê mô tả của từng cột

Tính các giá trị thống kê mô tả của các cột numeric bằng phương thức `describe()`

In [13]:
df.describe()

,Time,Temperature,Heat Index,Temperature Feels Like,Dew Point,Humidity,Wind Force,Wind Speed,Pressure,UV Index
count,1.739000e+04,17390.000000,17390.000000,17390.000000,17390.000000,17390.000000,17390.000000,17390.000000,17390.000000,17390.000000
mean,1.625183e+09,82.682691,88.648745,88.648745,74.017721,77.013823,5.458048,5.906095,29.760179,1.991777
std,9.068949e+06,5.317355,8.769664,8.769664,4.949475,16.297642,1.002710,3.696052,0.069052,3.048441
min,1.609434e+09,64.000000,64.000000,64.000000,46.000000,21.000000,0.370000,0.000000,29.520000,0.000000
25%,1.617342e+09,79.000000,82.000000,82.000000,72.000000,66.000000,6.000000,3.000000,29.700000,0.000000
50%,1.625194e+09,82.000000,88.000000,88.000000,75.000000,79.000000,6.000000,6.000000,29.760000,0.000000
75%,1.633030e+09,86.000000,95.000000,95.000000,77.000000,89.000000,6.000000,8.000000,29.820000,4.000000
max,1.640882e+09,99.000000,114.000000,114.000000,84.000000,100.000000,6.000000,31.000000,29.970000,15.000000


## 6. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Dùng `info()` để xem thuộc tính của từng cột

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Time                    17390 non-null  int64  
 1   Temperature             17390 non-null  int64  
 2   Heat Index              17390 non-null  float64
 3   Temperature Feels Like  17390 non-null  float64
 4   Dew Point               17390 non-null  float64
 5   Humidity                17390 non-null  float64
 6   Wind Force              17390 non-null  float64
 7   Wind Speed              17390 non-null  int64  
 8   Pressure                17390 non-null  float64
 9   UV Description          17390 non-null  object 
 10  UV Index                17390 non-null  int64  
 11  Condition               17390 non-null  object 
dtypes: float64(6), int64(4), object(2)
memory usage: 1.6+ MB


In [15]:
pd.set_option('display.max_columns', None)
df.head()

,Time,Temperature,Heat Index,Temperature Feels Like,Dew Point,Humidity,Wind Force,Wind Speed,Pressure,UV Description,UV Index,Condition
0,1609434000,75,75.0,75.0,61.0,61.0,6.0,8,29.85,Low,0,Partly Cloudy
1,1609435800,75,75.0,75.0,61.0,61.0,6.0,7,29.85,Low,0,Partly Cloudy
2,1609437600,75,75.0,75.0,61.0,61.0,6.0,7,29.82,Low,0,Partly Cloudy
3,1609439400,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Partly Cloudy
4,1609441200,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Partly Cloudy


Ta có thể thấy, cột `Time` nên là dữ liệu datetime, nhưng hiện giờ đang có kiểu dữ liệu numeric

Ta nên đưa giá trị 2 cột này về dạng datetime để có thể nhìn nhận và khám phá dễ hơn

In [16]:
def convert_time_column_to_datetime_type(df):
    if df['Time'].dtypes == 'int64':
        df['Time']= df['Time'].apply(datetime.fromtimestamp)
    return df


In [17]:
df = convert_time_column_to_datetime_type(df)
df.head()

,Time,Temperature,Heat Index,Temperature Feels Like,Dew Point,Humidity,Wind Force,Wind Speed,Pressure,UV Description,UV Index,Condition
0,2021-01-01 00:00:00,75,75.0,75.0,61.0,61.0,6.0,8,29.85,Low,0,Partly Cloudy
1,2021-01-01 00:30:00,75,75.0,75.0,61.0,61.0,6.0,7,29.85,Low,0,Partly Cloudy
2,2021-01-01 01:00:00,75,75.0,75.0,61.0,61.0,6.0,7,29.82,Low,0,Partly Cloudy
3,2021-01-01 01:30:00,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Partly Cloudy
4,2021-01-01 02:00:00,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Partly Cloudy


### Xem xét tập giá trị của các thuộc tính phân loại

Xem xét mỗi thuộc tính phân loại có bao nhiêu giá trị phân biệt bằng phương thức `set()`

In [18]:
set(df['UV Description'].to_list())

{'Extreme', 'High', 'Low', 'Moderate', 'Very High'}

In [19]:
len(set(df['UV Description'].to_list()))

5

In [20]:
set(df['Condition'].to_list())

{'Cloudy',
 'Fair',
 'Fog',
 'Haze',
 'Heavy Rain Shower',
 'Heavy Rain Shower / Windy',
 'Heavy T-Storm',
 'Heavy T-Storm / Windy',
 'Light Rain',
 'Light Rain Shower',
 'Light Rain Shower / Windy',
 'Light Rain with Thunder',
 'Mostly Cloudy',
 'Mostly Cloudy / Windy',
 'Partly Cloudy',
 'Partly Cloudy / Windy',
 'Rain',
 'Rain Shower',
 'Rain Shower / Windy',
 'Showers in the Vicinity',
 'T-Storm',
 'T-Storm / Windy',
 'Thunder',
 'Thunder in the Vicinity'}

In [21]:
len(set(df['Condition'].to_list()))

24

Nhận xét:
- Có nhiều loại điều kiện thời tiết được liệt kê (24 loại)
- Xuất hiện nhiều loại có thể xếp chung vào một nhóm

Để phù hợp với mục tiêu khám phá và phân tích dữ liệu, phân chia các loại vào các nhóm như sau:
- 'Cloudy'
- 'Fair'
- 'Fog / Haze'
- 'Rain'
- 'T-Storm'
- 'Thunder'

In [22]:
condition = {'Cloudy':'Cloudy',
             'Fair':'Fair',
             'Fog':'Fog / Haze',
             'Haze':'Fog / Haze',
             'Heavy Rain Shower':'Rain',
             'Heavy Rain Shower / Windy':'Rain',
             'Heavy T-Storm':'T-Storm',
             'Heavy T-Storm / Windy':'T-Storm',
             'Light Rain':'Rain',
             'Light Rain Shower':'Rain',
             'Light Rain Shower / Windy':'Rain',
             'Light Rain with Thunder':'Rain',
             'Mostly Cloudy':'Cloudy',
             'Mostly Cloudy / Windy':'Cloudy',
             'Partly Cloudy':'Cloudy',
             'Partly Cloudy / Windy':'Cloudy',
             'Rain':'Rain',
             'Rain Shower':'Rain',
             'Rain Shower / Windy':'Rain',
             'Showers in the Vicinity':'Rain',
             'T-Storm':'T-Storm',
             'T-Storm / Windy':'T-Storm',
             'Thunder':'Thunder',
             'Thunder in the Vicinity':'Thunder'}

In [23]:
df['Condition'] = df['Condition'].map(condition)
set(df['Condition'].to_list())

{'Cloudy', 'Fair', 'Fog / Haze', 'Rain', 'T-Storm', 'Thunder'}

## 7. Với mỗi cột có kiểu dữ liệu số, các giá trị phân bố như thế nào?

Với các cột có kiểu dữ liệu số, ta sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

Ta sẽ lưu kết quả vào DataFrame `num_col_info_df`, trong đó: 
- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng là: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"  

In [24]:
data = {
    "row_name" : ["missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"]
}
col_name = []
for i in df.columns:
    if (df[i].dtypes == 'float64') or (df[i].dtypes == 'int64'):
        col_name.append(i)

for i in col_name:
    _data = []
    missing_data = df[i].isna().sum()
    _data.append((missing_data / len(df[i]) * 100).round(1))
    _data.append(df[i].min())
    percentile = df[i].quantile([0.25,0.5,0.75])
    _data.append(percentile[0.25])
    _data.append(percentile[0.5])
    _data.append(percentile[0.75])
    _data.append(df[i].max())
    data[i] = _data
num_col_info_df = pd.DataFrame(data).set_index('row_name')

In [25]:
num_col_info_df

,Temperature,Heat Index,Temperature Feels Like,Dew Point,Humidity,Wind Force,Wind Speed,Pressure,UV Index
row_name,,,,,,,,,
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
min,64.0,64.0,64.0,46.0,21.0,0.37,0.0,29.52,0.0
lower_quartile,79.0,82.0,82.0,72.0,66.0,6.00,3.0,29.70,0.0
median,82.0,88.0,88.0,75.0,79.0,6.00,6.0,29.76,0.0
upper_quartile,86.0,95.0,95.0,77.0,89.0,6.00,8.0,29.82,4.0
max,99.0,114.0,114.0,84.0,100.0,6.00,31.0,29.97,15.0


## 8. Với mỗi cột có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

Thực hiện thống kê và lưu vào một dataframe với các dòng là đại diện cho các giá trị như sau:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu (missing_ratio).
- Số lượng các giá trị khác nhau (không xét giá trị thiếu) (num_values).
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu): dùng dictionary để lưu, key là giá trị, value là tỉ lệ % (value_ratios).

In [26]:
categorical_df=df.select_dtypes(exclude=['int64','float64','datetime64'])
data = {
    "row_name" : ["missing_ratio", "num_values", "value_ratios"]
}
col_names = list(categorical_df.columns)
for i in col_names:
    _data = []

    #Tính missing ratio
    missing_ratio = (categorical_df[i].isna().sum())/len(df[i]) * 100
    _data.append(missing_ratio)

    #Tính số lượng giá trị khác nhau
    num_values = categorical_df[i].dropna().nunique()
    _data.append(num_values)

    #Tính tỉ lệ của các giá trị
    dict_value_ratios ={}
    values = categorical_df[i].value_counts(normalize=True,sort=True)
    name_values = list(values.index)
    for k in range(len(name_values)):
        dict_value_ratios[name_values[k]] = round(values[k]*100,1)
    _data.append(dict_value_ratios)
    
    data[i] = _data

cat_col_info_df = pd.DataFrame(data).set_index('row_name')


In [27]:
cat_col_info_df

,UV Description,Condition
row_name,,
missing_ratio,0.0,0.0
num_values,5,6
value_ratios,"{'Low': 70.6, 'Moderate': 11.6, 'High': 9.0, '...","{'Cloudy': 73.9, 'Fair': 18.8, 'Rain': 5.7, 'T..."


In [28]:
df.head()

,Time,Temperature,Heat Index,Temperature Feels Like,Dew Point,Humidity,Wind Force,Wind Speed,Pressure,UV Description,UV Index,Condition
0,2021-01-01 00:00:00,75,75.0,75.0,61.0,61.0,6.0,8,29.85,Low,0,Cloudy
1,2021-01-01 00:30:00,75,75.0,75.0,61.0,61.0,6.0,7,29.85,Low,0,Cloudy
2,2021-01-01 01:00:00,75,75.0,75.0,61.0,61.0,6.0,7,29.82,Low,0,Cloudy
3,2021-01-01 01:30:00,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Cloudy
4,2021-01-01 02:00:00,73,73.0,73.0,61.0,65.0,4.0,6,29.82,Low,0,Cloudy


In [29]:
df.to_csv('weather-2021-preprocessed.csv', index=False)